In [6]:
import torch

In [7]:
import torch.nn as nn

In [60]:
class Decoder(nn.Module):
    def __init__(self, feature_size, embedding_size, vocab_size, hidden_size):
        super(Decoder,self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size,embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.fc = nn.Linear(hidden_size,vocab_size)
        self.init_h = nn.Linear(feature_size, hidden_size)  # linear layer to find initial hidden state of LSTMCell
        self.init_c = nn.Linear(feature_size, hidden_size)  # linear layer to find initial cell state of LSTMCell
        self.vocab_size = vocab_size
        
    def forward(self, features, captions, caption_lengths):
        batch_size = features.shape[0]
        sorted_caption_lengths, sort_ind = caption_lengths.squeeze(1).sort(dim=0, descending=True)
        print(features.shape)
        features = features[sort_ind]
        print(features.shape)
        captions = captions[sort_ind]
        caption_embeddings = self.embedding(captions)
        decode_lengths = sorted_caption_lengths - 1
        output = torch.zeros((batch_size,int(decode_lengths.max()),self.vocab_size))
        print(features.shape)
        hx = self.init_h(features) 
        print(hx.shape)
        cx = self.init_c(features)
        for t in range(int(decode_lengths.max())):
            batch_size_t = int((decode_lengths > t).sum())
            hx, cx = self.lstm(caption_embeddings[:batch_size_t,t,:], (hx, cx))
            raw_scores = self.fc(hx)
            output[:batch_size_t,t,:] = raw_scores       
        return output
        
        